<a href="https://colab.research.google.com/github/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/Chapter3_4_Going_Beyond_the_Basics_Detecting_Features_in_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3: Going Beyond the Basics: Detecting Features in Images
Hi, chào mọi người. Như trên tựa đề đã ghi, ở chương này, tụi mình sẽ đi xa hơn nữa, hơn cả mức cơ bản. Tìm hiểu sâu hơn với khác niệm mới trong mạng thần kinh.

<img src="https://gregorygundersen.com/image/cnns/41847277995.png" alt="Description" style="width:30%;">

Chương này khá dài nên mình sẽ tổng quan trước những cái mới mà chúng mình sẽ học nha:

1. Cách sử dụng **phương pháp tích chập** để trích xuất các **đặc trưng** trong ảnh. Thông qua đó ta hiểu được cách sử dụng các **lớp tích chập (Convolution layer)** để tăng hiệu quả học các bức ảnh. Thêm đó là phần **lớp gộp(Pooling layer)** giúp giảm thiểu số lượng tính toán, tuy nhiên vẫn giữ được các đặc trưng chính.
2. Phương pháp **gia tăng thêm dữ liệu (augmentation)** giúp làm giàu kho dữ liệu hơn, tăng dữ liệu trong quá trình huấn luyện giúp cải thiện mô hình.
3. Phương pháp **học chuyển giao (transfer learning)** giúp mô hình có thể tận dụng lại những kiến thức đã học được trước đó.
4. Ngoài ra chúng mình cũng sẽ ôn lại về **phân loại đa lớp (multi classification)** cũng như tìm hiểu sơ qua về phương pháp **loại bỏ ngẫu nhiên (dropout)** giúp cải thiện mô hình, tránh được tình trạng **quá khớp (overfitting)**.

*Lưu ý: Khuyến khích mọi người sử dụng Colab hãy chuyển sang T4 GPU ở chương này nha vì việc huấn luyện mô hình sử dụng các lớp tích chập tốn khá nhiều thời gian tính toán.*

# 4. Ôn tập về **phân loại đa lớp (multi classification)** và tìm hiểu về cơ chế **loại bỏ ngẫu nhiên (dropout)**.

Nếu mọi người nhớ lại thì thật ra tụi mình đã từng làm về bài toán phân loại đa lớp ở chương 2 với bộ dữ liệu [Fashion Mnist](https://keras.io/api/datasets/fashion_mnist/) rồi ý. Nếu mọi người chưa coi thì có thể nhấp vào coi thử [Chương 2](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/Chapter2_Introduction_to_Computer_Vision.ipynb) nha. Hiểu đơn giản là thay vì xác định 2 đối tượng (phân loại nhị phân) thì giờ đây ta sẽ xác định nhiều đối tượng hơn.

Chính vì sự khác biệt ấy nên khi xây dựng mô hình thì tụi mình cũng có chút khác biệt nha. Điểm này chính là mấu chốt chính để xác định luôn nha.

- Với bài toán **phân loại nhị phân (binary)**, lớp tuyến tính cuối cùng hay lớp kết nối đầy đủ **số đơn vị đầu ra là 1** với hàm kích hoạt là **sigmoid**. Như vậy đầu ra của chúng là tỷ lệ trong khoảng từ 0 đến 100%. Chúng ta đặt một **ngưỡng threshold** để phân định, nếu thấp hơn thì là nhãn 0 và ngược lại là 1. Ngoài ra hàm mất mát (loss) của chúng ta sẽ là **Binary cross entropy**.
> Dense(1, activation = 'relu')

  > model.compile(loss = 'binary_crossentropy', ...)

- Với bài toán **phân loại đa lớp**, lớp cuối cùng có số **đơn vị đầu ra tương ứng với số nhãn** của bạn và hàm kích hoạt là **softmax**. Đầu ra của chúng là một mảng hay vector với kích thước là số nhãn phân loại, mỗi giá trị sẽ tương ứng với tỷ lệ đối tượng thuộc nhãn ở vị trí index đó.
> Ví dụ: ta cần phân biệt 4 nhãn, đầu ra của mô hình sẽ là **[0.1, 0.2, 0.5, 0.2]**, lấy tỷ lệ cao nhất tức đối tượng đó thuộc nhãn 2 (tính theo chỉ mục từ 0)

  > Dense(n_output, activation = 'softmax')

  > model.compile(loss = 'categorical_crossentropy', ...)

### Ôn lý thuyết nhiều chắc mọi người cũng chán ha, tụi mình sẽ đi qua thực hành luôn một bài toán phân lớp khác nha.
Lần này bài toán của tụi mình sẽ là